# **Loading Dataset**

---



In [41]:
import kagglehub

path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

Using Colab cache for faster access to the 'imdb-dataset-of-50k-movie-reviews' dataset.


In [42]:
import pandas as pd

dataset = pd.read_csv(path + "/IMDB Dataset.csv")
dataset.shape

(50000, 2)

In [43]:
df = dataset.iloc[:10000].copy()
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [44]:
df['sentiment'].value_counts()

,count
sentiment,
positive,5028
negative,4972


In [45]:
df.isnull().sum()

,0
review,0
sentiment,0


In [46]:
df.duplicated().sum()

np.int64(17)

In [47]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

np.int64(0)

# **Text Preprocessing**

---



In [48]:
import re

def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [49]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [50]:
df['review'] = df['review'].apply(remove_tags)
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [51]:
df['review'] = df['review'].apply(lambda x:x.lower())
df['review'][0]

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [52]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
sw_list = stopwords.words('english')

df['review'] = df['review'].apply(
        lambda x: [item for item in x.split() if item not in sw_list]).apply(
        lambda x:" ".join(x)
        )


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
df['review'][0]

"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. hardcore, classic use word.it called oz nickname given oswald maximum security state penitentary. focuses mainly emerald city, experimental section prison cells glass fronts face inwards, privacy high agenda. em city home many..aryans, muslims, gangstas, latinos, christians, italians, irish more....so scuffles, death stares, dodgy dealings shady agreements never far away.i would say main appeal show due fact goes shows dare. forget pretty pictures painted mainstream audiences, forget charm, forget romance...oz mess around. first episode ever saw struck nasty surreal, say ready it, watched more, developed taste oz, got accustomed high levels graphic violence. violence, injustice (crooked guards who'll sold nickel, inmates who'll kill order g

# **Training**

---



In [54]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [55]:
X.head()

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."


In [56]:
y.head()

,sentiment
0,positive
1,positive
2,positive
3,negative
4,positive


In [57]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y = encoder.fit_transform(y)
y

array([1, 1, 1, ..., 0, 0, 1])

In [58]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **Bag of Words, N-grams & TF-IDF Splits**

---



In [59]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=3000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [60]:
from sklearn.feature_extraction.text import CountVectorizer

cv2 = CountVectorizer(ngram_range=(1,2), max_features=3000)

X_train_ngram = cv2.fit_transform(X_train['review']).toarray()
X_test_ngram = cv2.transform(X_test['review']).toarray()

X_train_ngram

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=3000)

X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review']).toarray()

X_train_tfidf

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.06189641, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [62]:
!pip install gensim

In [63]:
import gensim
from gensim.utils import simple_preprocess
from nltk import sent_tokenize

new_df = pd.DataFrame()
new_df['tokens'] = df['review'].apply(lambda x: simple_preprocess(x))

w2v_model = gensim.models.Word2Vec(window=5, min_count=1)
w2v_model.build_vocab(new_df['tokens'])
w2v_model.train(new_df['tokens'], total_examples=w2v_model.corpus_count, epochs=w2v_model.epochs)

(5955953, 6186875)

In [64]:
import numpy as np
def document_vector(text, model):
    doc = [word for word in text if word in w2v_model.wv]
    if len(text) == 0:
        return np.zeros(w2v_model.vector_size)
    return np.mean(w2v_model.wv[text], axis=0)

X_w2v = np.array([document_vector(text, w2v_model) for text in new_df['tokens']])
X_train_w2v, X_test_w2v = train_test_split(X_w2v, test_size=0.2, random_state=42)

# **Naive Bayes**

---



In [65]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

In [66]:
nb1 = MultinomialNB()
nb1.fit(X_train_bow, y_train)
nb_pred_bow = nb1.predict(X_test_bow)
print(f"Bag of Words Accuracy: {accuracy_score(y_test, nb_pred_bow)}")
print(f"Bag of Words Confusion Matrix: \n{confusion_matrix(y_test, nb_pred_bow)}")

Bag of Words Accuracy: 0.8582874311467201
Bag of Words Confusion Matrix: 
[[853 132]
 [151 861]]


In [67]:
nb2 = MultinomialNB()
nb2.fit(X_train_ngram, y_train)
nb_pred_ngram = nb2.predict(X_test_ngram)
print(f"N-grams Accuracy: {accuracy_score(y_test, nb_pred_ngram)}")
print(f"N-grams Confusion Matrix: \n{confusion_matrix(y_test, nb_pred_ngram)}")

N-grams Accuracy: 0.8582874311467201
N-grams Confusion Matrix: 
[[843 142]
 [141 871]]


In [68]:
nb3 = MultinomialNB()
nb3.fit(X_train_tfidf, y_train)
nb_pred_tfidf = nb3.predict(X_test_tfidf)
print(f"TF-IDF Accuracy: {accuracy_score(y_test, nb_pred_tfidf)}")
print(f"TF-IDF Confusion Matrix: \n{confusion_matrix(y_test, nb_pred_tfidf)}")

TF-IDF Accuracy: 0.8703054581872809
TF-IDF Confusion Matrix: 
[[858 127]
 [132 880]]


# **Random Forest Classifier**

---



In [69]:
from sklearn.ensemble import RandomForestClassifier

In [70]:
rf1 = RandomForestClassifier()
rf1.fit(X_train_bow, y_train)
rf_pred_bow = rf1.predict(X_test_bow)
print(f"Bag of Words Accuracy: {accuracy_score(y_test, rf_pred_bow)}")
print(f"Bag of Words Confusion Matrix: \n{confusion_matrix(y_test, rf_pred_bow)}")

Bag of Words Accuracy: 0.8377566349524287
Bag of Words Confusion Matrix: 
[[836 149]
 [175 837]]


In [71]:
rf2 = RandomForestClassifier()
rf2.fit(X_train_ngram, y_train)
rf_pred_ngram = rf2.predict(X_test_ngram)
print(f"N-grams Accuracy: {accuracy_score(y_test, rf_pred_ngram)}")
print(f"N-grams Confusion Matrix: \n{confusion_matrix(y_test, rf_pred_ngram)}")

N-grams Accuracy: 0.842764146219329
N-grams Confusion Matrix: 
[[837 148]
 [166 846]]


In [72]:
rf3 = RandomForestClassifier()
rf3.fit(X_train_tfidf, y_train)
rf_pred_tfidf = rf3.predict(X_test_tfidf)
print(f"TF-IDF Accuracy: {accuracy_score(y_test, rf_pred_tfidf)}")
print(f"TF-IDF Confusion Matrix: \n{confusion_matrix(y_test, rf_pred_tfidf)}")

TF-IDF Accuracy: 0.8417626439659489
TF-IDF Confusion Matrix: 
[[847 138]
 [178 834]]


In [73]:
rf4 = RandomForestClassifier()
rf4.fit(X_train_w2v, y_train)
rf_pred_w2v = rf4.predict(X_test_w2v)
print(f"Word2Vec Accuracy: {accuracy_score(y_test, rf_pred_w2v)}")
print(f"Word2Vec Confusion Matrix: \n{confusion_matrix(y_test, rf_pred_w2v)}")

Word2Vec Accuracy: 0.7876815222834251
Word2Vec Confusion Matrix: 
[[782 203]
 [221 791]]
